In [ ]:
# Mount Google Drive to access uploaded data
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install torcheval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 6.0 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import pandas
import cv2 as cv
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torcheval.metrics import BinaryF1Score, BinaryRecall, BinaryPrecision, BinaryAccuracy
from tqdm.auto import tqdm
import time

In [ ]:
model_resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 224MB/s]


In [ ]:
batch_size = 128
random_seed = 8
torch.manual_seed(random_seed)

In [ ]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
dataset = torchvision.datasets.ImageFolder(root="/content/gdrive/MyDrive/Sem_VII_project/WCEBleedGen", transform=data_transforms)
print(dataset.classes)

['bleeding', 'non-bleeding']


In [ ]:
train_len = int(0.7 * len(dataset))
val_len = int(0.1 * len(dataset))
test_len = len(dataset) - (train_len + val_len)

In [ ]:
train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

In [ ]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(val_ds, batch_size = batch_size)
test_dl = torch.utils.data.DataLoader(test_ds)

In [ ]:
# for param in model_resnet.parameters():
#  param.requires_grad = False
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(in_features=num_ftrs, out_features=1)

In [ ]:
if not torch.cuda.is_available():
  device = 'cpu'
else:
  device = 'cuda'
print(device)

cpu


In [ ]:
model_resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
metric = BinaryF1Score(device=device)
prec = BinaryPrecision(device=device)
recall = BinaryRecall(device=device)
acc = BinaryAccuracy(device=device)

In [ ]:
def train(epochs, model):
  optimizer = torch.optim.Adadelta(model.parameters())
  loss_func = nn.BCEWithLogitsLoss(reduction='mean')
  for epoch in range(epochs):
    time.sleep(1)
    model.train()
    batch=0
    running_loss = 0
    for _, (inputs, labels) in enumerate(tqdm(train_dl)):
      inputs, labels = inputs.cuda(), labels.cuda()
      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        # preds = torch.round(outputs.sigmoid()).squeeze()
        loss = loss_func(outputs, labels.float().unsqueeze(1))
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"[+] epoch {epoch+1}/{epochs} running loss: {running_loss}")
    model.eval()
    with torch.inference_mode():
      valid_loss = 0
      total_acc = 0
      len_preds = 0
      for inputs, labels in valid_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        preds = outputs.sigmoid().squeeze()
        loss = F.binary_cross_entropy(torch.sigmoid(outputs), labels.float().unsqueeze(1))
        valid_loss += loss.item()
        metric.update(preds, labels)
        prec.update(preds, labels)
        recall.update(preds, labels)
        acc.update(preds, labels)
      print(f"[-] total validation loss for epoch {epoch}: {valid_loss} accuracy: {acc.compute()} precision: {prec.compute()} recall: {recall.compute()} F1Score: {metric.compute()}")


In [ ]:
train(4, model_resnet)

  0%|          | 0/29 [00:00<?, ?it/s]

[+] epoch 1/4 running loss: 11.32929590344429
[-] total validation loss for epoch 0: 1.2530866414308548 accuracy: 0.8891013264656067 precision: 1.0 recall: 0.7867646813392639 F1Score: 0.8806584477424622


  0%|          | 0/29 [00:00<?, ?it/s]

[+] epoch 2/4 running loss: 3.113548696041107
[-] total validation loss for epoch 1: 0.3324096240103245 accuracy: 0.9340344071388245 precision: 0.99790358543396 recall: 0.875 F1Score: 0.9324191808700562


  0%|          | 0/29 [00:00<?, ?it/s]

[+] epoch 3/4 running loss: 1.1644036332145333
[-] total validation loss for epoch 2: 6.475286960601807 accuracy: 0.8597832918167114 precision: 0.9685534834861755 recall: 0.7549019455909729 F1Score: 0.8484848737716675


  0%|          | 0/29 [00:00<?, ?it/s]

[+] epoch 4/4 running loss: 1.0354341899510473
[-] total validation loss for epoch 3: 0.17175074480473995 accuracy: 0.8905353546142578 precision: 0.9777530431747437 recall: 0.8079044222831726 F1Score: 0.8847509026527405


In [ ]:
import torch

# ...

# Train your model as you did before

# Save the trained model
torch.save(model_resnet.state_dict(), '/content/gdrive/MyDrive/Sem_VII_project/model_resnet.pth')


In [ ]:
def evaluate(model, test_data, eval_metric):
  model.eval()
  with torch.inference_mode():
    valid_loss = 0
    total_acc = 0
    len_preds = 0
    for inputs, labels in test_data:
      inputs = inputs.cuda()
      labels = labels.cuda()
      outputs = model(inputs)
      preds = torch.round(outputs.sigmoid()).squeeze(1)
      eval_metric.update(preds, labels)
      prec.update(preds, labels)
      recall.update(preds, labels)
      loss = F.binary_cross_entropy_with_logits(outputs, labels.float().unsqueeze(1))
      valid_loss += loss.item()
      total_acc += torch.tensor(torch.sum(preds == labels).item())
      len_preds += len(preds)
    print(f"[*] total binary cross entropy loss: {valid_loss} accuracy: {total_acc/len_preds} F1Score: {metric.compute()} BinaryRecall: {recall.compute()} Precision: {prec.compute()}")
    eval_metric.reset()
    prec.reset()
    recall.reset()


In [ ]:
metric.reset()
prec.reset()
recall.reset()
acc.reset()
evaluate(model_resnet, test_dl, metric)

[*] total binary cross entropy loss: 35.90907765463201 accuracy: 0.9866412281990051 F1Score: 0.9860557317733765 BinaryRecall: 0.9821428656578064 Precision: 0.9900000095367432


In [ ]:
from PIL import Image
import os

In [ ]:
path = "/content/gdrive/MyDrive/Sem_VII_project/Auto-WCEBleedGenTest/TestDataset1"
d = {0.:0,
     1.:0}
transform = data_transforms
for i in os.listdir(path):
  img = Image.open(os.path.join(path, i))
  img = transform(img)
  img = img.to(device)
  img = torch.unsqueeze(img, dim=0)
  with torch.inference_mode():
    output = model_resnet(img).squeeze()
    pred = torch.round(output.sigmoid())
    print(i, pred, output.sigmoid())
    d[pred.item()] += 1


print(d[0.], d[1.])


In [ ]:
path = "/content/gdrive/MyDrive/Sem_VII_project/Auto-WCEBleedGenTest/TestDataset2"
d = {0.:0,
     1.:0}
transform = data_transforms
for i in os.listdir(path):
  img = Image.open(os.path.join(path, i))
  img = transform(img)
  img = img.to(device)
  img = torch.unsqueeze(img, dim=0)
  with torch.inference_mode():
    output = model_resnet(img).squeeze()
    pred = torch.round(output.sigmoid())
    print(i, pred, output.sigmoid())
    d[pred.item()] += 1


print(d[0.], d[1.])
